In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 3.7 MB/s 


In [ ]:
!pip install onnx2keras --quiet

     |████████████████████████████████| 13.1 MB 4.9 MB/s 


In [ ]:
!pip install tensorflow tf2onnx --quiet

     |████████████████████████████████| 442 kB 5.4 MB/s 


In [ ]:
!pip install onnxruntime --quiet

     |████████████████████████████████| 4.9 MB 5.1 MB/s 
     |████████████████████████████████| 46 kB 2.6 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 


In [38]:
import os, cv2,onnx
import numpy as np
#import onnxruntime
import tf2onnx
import onnxruntime as rt

In [34]:
def preprocess(image):

  #preprocess image
     
    img = cv2.imread(image).astype("float32") #cvt to greyscale by 0....Ada path
    img = cv2.resize(img, (256,256))
    img = img.copy()/255
    img = np.expand_dims(img, axis=0)

    return img

In [35]:
def onnx_prediction(model, image):
  x = preprocess(image)

  providers = ['CPUExecutionProvider']
  m = rt.InferenceSession(model, providers=providers)
  onnx_pred = m.run([], {"input": x})

  predictions = (onnx_pred[0]> 0.5).astype("int32")
  predictions = predictions.reshape(1,-1)[0]
  return predictions[0]

#print('ONNX Predicted:', decode_predictions(onnx_pred[0], top=3)[0])

# make sure ONNX and keras have the same results
#np.testing.assert_allclose(preds, onnx_pred[0], rtol=1e-5)

In [36]:
#import onnx
#from onnx2keras import onnx_to_keras

# Load ONNX model
#onnx_model = onnx.load('/content/drive/MyDrive/Covid19/model.onnx')

# Call the converter (input - is the main model input name, can be different for your model)
#k_model = onnx_to_keras(onnx_model, ['spec'])

In [39]:
onnx_prediction('/content/drive/MyDrive/Covid19/model.onnx', "/content/Normal-2.png")

1

In [ ]:
def onnx_prediction_batch(model, foldername):

  #data_size = len([entry for entry in os.listdir(foldername) if os.path.isfile(os.path.join(foldername, entry))]) + len([entry for entry in os.listdir(covid_path) if os.path.isfile(os.path.join(covid_path, entry))])

  X = np.zeros((600,256,256,3))

 # normal_path = r'/content/drive/MyDrive/Covid19/Covid Dataset/Normal_Large'
  #for i in range(len([entry for entry in os.listdir(normal_path) if os.path.isfile(os.path.join(normal_path, entry))])):
  for image in os.listdir(foldername):
    image_path = os.path.join(foldername, image)
    img = cv2.imread(image_path) #cvt to greyscale by 0....Ada path
    img = cv2.resize(img, (256,256))
    img = img.copy()/255
    X[i] = np.expand_dims(img, axis=0)

  providers = ['CPUExecutionProvider']
  m = rt.InferenceSession(output_path, providers=providers)
  onnx_pred = m.run([], {"input": X})

  predictions = (onnx_pred> 0.5).astype("int32")
  predictions = predictions.reshape(1,-1)[0]
  return predictions
  #preprocess image
  